In [3]:
import ipywidgets as widgets
import ipyvuetify as v
import ipysheet
import global_string_gen as generate_string
from global_filegen import generate_file
import variable_recognition as vr
import numpy as np
from IPython.display import display, Markdown

Result = []

def analyse_click(widget, event, data):
    
    analyse_but.loading=True
    analyse_but.disabled=True
    
    in_str = function.v_model
    result = generate_string.total_parse(in_str)
    n = 0
    for g in result:
        for c in g[1]:
            n += len(c.var_in)
            n += len(c.var_out)
    
    headers = ['Group Name','Component Name','Variable Detected','Variable Name','Input/Output','Units','Default Value']

    sheet = ipysheet.sheet(columns=7, rows=n, row_headers=False, column_headers=headers)
    
    i = 0
    for g in result:
        for c in g[1]:
            for var_in in c.var_in:
                ipysheet.cell(i,0,c.group)
                ipysheet.cell(i,1,c.name)
                ipysheet.cell(i,2,var_in[0])
                ipysheet.cell(i,3,var_in[1])
                ipysheet.cell(i,4,'input', background_color='#8EFF9B')
                ipysheet.cell(i,5,'None')
                ipysheet.cell(i,6,'np.nan')
                i += 1
            for var_out in c.var_out:
                ipysheet.cell(i,0,c.group)
                ipysheet.cell(i,1,c.name)
                ipysheet.cell(i,2,var_out[0])
                ipysheet.cell(i,3,var_out[1])
                ipysheet.cell(i,4,'output', background_color='#FFB48E')
                ipysheet.cell(i,5,'None')
                ipysheet.cell(i,6,'')
                i += 1
                
    analyse_but.loading=False
    analyse_but.children=['Analysis done']
    vb.children=(list(vb.children) + [sheet, print_but, gen_but])
    
    global Result
    Result = result
    

def print_click(widget, event, data):
    
    #Function called when the button 'print code' is pressed
    
    print_but.loading=True
    print_but.disabled=True
    
    sheet = ipysheet.current()
    arr = np.array([])
    arr = ipysheet.to_array(sheet)
    
    global Result
    
    result = Result
    
    i = 0
    for g in result:
        for c in g[1]:
            j = 0
            for var_in in c.var_in:
                c.group = arr[i,0]
                c.name = arr[i,1]
                var_in[0] = arr[i,2]
                var_in[1] = arr[i,3]
                c.units_i[j][0] = arr[i,5]
                c.units_i[j][1] = arr[i,6]
                i += 1
                j += 1
                
            k = 0  
            for var_out in c.var_out:
                c.group = arr[i,0]
                c.name = arr[i,1]
                var_out[0] = arr[i,2]
                var_out[1] = arr[i,3]
                c.units_o[k] = arr[i,5]
                i += 1
                k += 1
                
    print_but.loading=False
    print_but.children=['Code printed']
    
    s = generate_string.gen_string(result)
    display(Markdown("```python\n" + s + "\n```"))

    
def gen_click(widget, event, data):
    
    # Function called when the button 'generate file' is pressed
    
    gen_but.loading=True
    gen_but.disabled=True
    
    sheet = ipysheet.current()
    arr = np.array([])
    arr = ipysheet.to_array(sheet)
    
    global Result
    
    result = Result
    # Modifying result to take into account changes made to the sheet
    i = 0
    for g in result:
        for c in g[1]:
            j = 0
            for var_in in c.var_in:
                c.group = arr[i,0]
                c.name = arr[i,1]
                var_in[0] = arr[i,2]
                var_in[1] = arr[i,3]
                c.units_i[j][0] = arr[i,5]
                c.units_i[j][1] = arr[i,6]
                i += 1
                j += 1
            k = 0
            for var_out in c.var_out:
                c.group = arr[i,0]
                c.name = arr[i,1]
                var_out[0] = arr[i,2]
                var_out[1] = arr[i,3]
                c.units_o[k] = arr[i,5]
                i += 1
                k += 1
                
    gen_but.loading=False
    gen_but.children=['File generated ']
    
    generate_file(result)
  
    
analyse_but = v.Btn(children=['Analyse'], color='blue lighten-1')
analyse_but.on_event('click', analyse_click)

print_but = v.Btn(children=['Print code'], color='orange lighten-2')
print_but.on_event('click', print_click)

gen_but = v.Btn(children=['Generate file'], color="green lighten-2")
gen_but.on_event('click', gen_click)


function = v.Textarea(
v_model= '# Component1\ny = x + 1', 
label='Equations',
clearable=True,
rounded=True,
auto_grow=False,
rows=15, 
background_color="blue lighten-4" 
)

vb = widgets.VBox(children=[function, analyse_but])

vb


In [ ]:
"""
---Test Text---

# Component1

x = 2*y + 3*z
a = b + c

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y))


"""

In [ ]:
"""
---Test Text---
## Group1

# Component1

x = 2*y + 3*z
a = b + c

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y))

## Group2

# Component3

x = 2*y + 3*z
a = b + c

# Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""